In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time

import numpy as np
from multiprocess import Pool

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD585/MD585_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_anchor.txt
File does not exist: 

In [2]:
stack = 'MD591'
first_sec, last_sec = metadata_cache['section_limits'][stack]

rm -rf /shared/CSHL_data_processed/MD591/MD591_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD591
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD591/MD591_cropbox.txt /shared/CSHL_data_processed/MD591/MD591_cropbox.txt


Child returned 0
Child returned 0
0.48 seconds.


In [3]:
classifier_id = 30

In [ ]:
if classifier_id == 12:
    available_classifiers = {2: DataManager.load_classifiers(setting=2),
                             10: DataManager.load_classifiers(setting=10)}
else:
    available_classifiers = {classifier_id: DataManager.load_classifiers(setting=classifier_id)}

In [15]:
def clf_predict(stack, sec, model_name='Inception-BN'):
    
    if is_invalid(stack=stack, sec=sec):
        return

    try:
        features = DataManager.load_dnn_features(stack=stack, model_name=model_name, section=sec)
    except Exception as e:
        sys.stderr.write('%s\n' % e.message)
        return

    actual_setting = resolve_actual_setting(setting=classifier_id, stack=stack, sec=sec)
    clf_allClasses_ = available_classifiers[actual_setting]

    for structure, clf in clf_allClasses_.iteritems():

        probs = clf.predict_proba(features)[:, clf.classes_.tolist().index(1.)]
        output_fn = DataManager.get_sparse_scores_filepath(stack=stack, structure=structure, 
                                                           setting=actual_setting, sec=sec)
        create_parent_dir_if_not_exists(output_fn)
        bp.pack_ndarray_file(probs, output_fn)

In [16]:
clf_predict(stack=stack, sec=203)

In [ ]:
t = time.time()

pool = Pool(NUM_CORES/2)
pool.map(lambda sec: clf_predict(stack=stack, sec=sec), range(200, 201))
# pool.map(lambda sec: clf_predict(stack=stack, sec=sec), range(first_sec, last_sec+1))
pool.close()
pool.join()

sys.stderr.write('Classifier predict: %.2f\n' % (time.time()-t)) # 30s each section, single process; 